# Chapter 6. Pipelines.
# Part 1. Basis.

Notes:

1) Data Scaling should be applied only to the training folds, thus cross-validation should be done BEFORE scaling in terms of using pipeline.

2) There could be any number of stages in pipeline with the only requirement to keep all stages (except last one) to have method 'transform' in a such way that new transformed data could be used for next stage.

#### Setting up:

In [11]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#-----dataset preparing
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)
#scaling
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

#-----model initialization and building
svm = SVC().fit(X_train_scaled, y_train)
X_test_scaled = scaler.transform(X_test)

print('Test accuracy: {}'.format(svm.score(X_test_scaled, y_test)))

Test accuracy: 0.972027972027972


## - Building Pipelines.

Corteges made of preferred name and used model are representing stages.

In [12]:
from sklearn.pipeline import Pipeline

#initialization and building pipeline model:
pipe = Pipeline([('scaler', MinMaxScaler()), ('svm', SVC())])
pipe.fit(X_train, y_train)

#accuracy:
print('Test accuracy: {}'.format(pipe.score(X_test, y_test)))

Test accuracy: 0.972027972027972


There's also a handy way to build pipeline via 'make_pipeline'. It's the same thing but the stage's name is assigned automatically:

In [16]:
from sklearn.pipeline import make_pipeline

#standard syntax:
pipe_long = ([('scaler', MinMaxScaler()), ('svm', SVC(C=100))])
#'make_pipeline' syntax:
pipe_short = make_pipeline(MinMaxScaler(), SVC(C=100))

#assigned stage names within corteges:
print('Pipelines stages:\n{}'.format(pipe_short.steps))

Pipelines stages:
[('minmaxscaler', MinMaxScaler()), ('svc', SVC(C=100))]


## - Pipelines with GridSearchCV

Using GridSearchCV with pipelines is the same as usual (put the model inside GridSearchCV) but the grid itself should be assigned via specific syntax:

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
              'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)

print('Best cross-validation accuracy: {}'.format(grid.best_score_))
print('Test accuracy: {}'.format(grid.score(X_test, y_test)))
print('Best params: {}'.format(grid.best_params_))

Best cross-validation accuracy: 0.9812311901504789
Test accuracy: 0.972027972027972
Best params: {'svm__C': 1, 'svm__gamma': 1}


## - Stage's atributes
To see stage's model atributes it should be used 'named_steps' which is a dictionary of names attached to models:

In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pipe = make_pipeline(StandardScaler(), PCA(n_components=2), StandardScaler())

pipe.fit(cancer.data)
components = pipe.named_steps['pca'].components_

print('Steps: {}\n'.format(pipe.named_steps))
print('PCA components shape: {}'.format(components.shape))

Steps: {'standardscaler-1': StandardScaler(), 'pca': PCA(n_components=2), 'standardscaler-2': StandardScaler()}

PCA components shape: (2, 30)
